In [32]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

In [1]:
# import requests
# from bs4 import BeautifulSoup


# url="https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt"

# response = requests.get(url)

# if response.status_code==200:
#     soup = BeautifulSoup(response.content,"html.parser")
#     text= soup.get_text()
#     with open("output.txt",'w',encoding='utf-8') as f:
#         f.write(text)
        
#     print("text has been written")
    
# else:
#     print("failed to write the content",requests.status_codes)
    

In [2]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM


filepath = "./output.txt"
try:
    with open(filepath,'r',encoding='utf-8') as f:
        
        text = f.read().lower()
        print("the file was read")
        
except FileNotFoundError:
    print("the file was not found")
    
except Exception as e :
    print(f"an error has occured: {e}")

the file was read


In [3]:
te=text[300:8000000]
character=sorted(set(te))

In [4]:
# character

In [5]:
char_to_index= dict((c,i)for i,c in enumerate(character))

index_to_char= dict((i,c)for i,c in enumerate(character))

In [6]:
seq_lenght=40
step_size=3
sentence=[]
next_character=[]

In [7]:
for i in range(0,len(te)-seq_lenght,step_size):
    sentence.append(te[i:i+seq_lenght])b
    
    next_character.append(te[i+seq_lenght])

In [10]:
for s, n in zip(sentence[:10], next_character[:10]):
    print(f"Sequence: '{s}' -> Next Character: '{n}'")

Sequence: ' kill him, and we'll have corn at our ow' -> Next Character: 'n'
Sequence: 'll him, and we'll have corn at our own p' -> Next Character: 'r'
Sequence: 'him, and we'll have corn at our own pric' -> Next Character: 'e'
Sequence: ', and we'll have corn at our own price.
' -> Next Character: 'i'
Sequence: 'nd we'll have corn at our own price.
is'' -> Next Character: 't'
Sequence: 'we'll have corn at our own price.
is't a' -> Next Character: ' '
Sequence: 'll have corn at our own price.
is't a ve' -> Next Character: 'r'
Sequence: 'have corn at our own price.
is't a verdi' -> Next Character: 'c'
Sequence: 'e corn at our own price.
is't a verdict?' -> Next Character: '
'
Sequence: 'orn at our own price.
is't a verdict?

a' -> Next Character: 'l'


In [19]:
df_sentence= pd.DataFrame([list(s) for s in sentence])
df_next_char= pd.DataFrame(next_character,columns=['next_character'])

In [40]:
num_chars = len(character)

In [17]:
df_sentence

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,,k,i,l,l,,h,i,m,",",...,,a,t,,o,u,r,,o,w
1,l,l,,h,i,m,",",,a,n,...,,o,u,r,,o,w,n,,p
2,h,i,m,",",,a,n,d,,w,...,r,,o,w,n,,p,r,i,c
3,",",,a,n,d,,w,e,',l,...,w,n,,p,r,i,c,e,.,\n
4,n,d,,w,e,',l,l,,h,...,p,r,i,c,e,.,\n,i,s,'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371680,e,,s,l,e,e,p,-,-,d,...,w,h,i,l,e,s,,t,h,o
371681,l,e,e,p,-,-,d,i,e,",",...,l,e,s,,t,h,o,u,,a
371682,p,-,-,d,i,e,",",,r,a,...,,t,h,o,u,,a,r,t,
371683,d,i,e,",",,r,a,t,h,e,...,o,u,,a,r,t,,w,a,k


In [30]:
x_encoded = pd.get_dummies(df_sentence.apply(pd.Series).stack()).groupby(level=0).sum().reset_index(drop=True)
y_encoded = pd.get_dummies(df_next_char['next_character'])


In [31]:
x = x_encoded.values.astype(np.bool_)
y = y_encoded.values.astype(np.bool_)

In [41]:
model=Sequential()
model.add(LSTM(128,input_shape=(seq_lenght,num_chars)))
model.add(Dense(num_chars))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 128)            │        86,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 39)             │         5,031 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 39)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 91,047 (355.65 KB)

 Trainable params: 91,047 (355.65 KB)

 Non-trainable params: 0 (0.00 B)

In [44]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.01))

model.fit(x, y, batch_size=256, epochs=4)

Epoch 1/4


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("sequential_6_1/Cast:0", shape=(None, 39), dtype=float32). Expected shape (None, 40, 39), but input has incompatible shape (None, 39)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 39), dtype=bool)
  • training=True
  • mask=None